In [2]:
#import packages
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import Imputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [3]:
#import data from excel
traindata = pd.read_csv("/Users/justinkoehn/Documents/2021-train.csv")


In [4]:
#subset is_swing column from training data
isswing=traindata["is_swing"]

In [6]:
#create test set function
def split_train_test(data, test_ratio):
    shuffled_indices = np.random.permutation(len(data))
    test_set_size = int(len(data) * test_ratio) 
    test_indices = shuffled_indices[:test_set_size] 
    train_indices = shuffled_indices[test_set_size:]
    return data.iloc[train_indices], data.iloc[test_indices]

In [8]:
#select training data based on most predictive features
traindata_numeric=traindata[["balls","strikes","outs", "spin_rate","extension","plate_height","vert_break", "horz_break","vert_approach_angle","x55"]]
traindata_numeric['is_swing']=isswing


/Users/justinkoehn/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [10]:
#remove null values
traindata_numeric=traindata_numeric.dropna()

In [14]:
#split dataset based off .2 ratio
train_set, test_set = split_train_test(traindata_numeric, 0.2) 
print(len(train_set), "train +", len(test_set), "test")

824736 train + 206184 test


In [15]:
#divide training set into x and y 
is_swing_label = train_set["is_swing"]
features_train=train_set.drop(["is_swing"],axis=1)


In [18]:
#Scale Data
scaler = StandardScaler() 
scaler.fit(features_train)
features_train = scaler.transform(features_train)


In [20]:
#Fit logistic regression model
log_reg_classifier = LogisticRegression()
log_reg_classifier.fit(features_train,is_swing_label)

/Users/justinkoehn/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [21]:
#divide test set into x and y 
is_swing_label_test = test_set["is_swing"]
features_test=test_set.drop(["is_swing"],axis=1)


In [22]:
#create swing/no swing predictions based on selcted features 
predictions=log_reg_classifier.predict(features_test)

In [24]:
#find accuracry score from model based on predictions and ground truth
accuracy=metrics.accuracy_score(predictions, is_swing_label_test)
accuracy

In [26]:
#read in test data
testdata = pd.read_csv("/Users/justinkoehn/Downloads/2021-test (1).csv")

In [28]:
#select data that corresponds to model
testdata_numeric=testdata[["balls","strikes","outs", "spin_rate","extension","plate_height","vert_break", "horz_break","vert_approach_angle","x55"]]


In [29]:
#drop null data
testdata_numeric=testdata_numeric.dropna()

In [30]:
#scale corresponding test data
features_test = scaler.transform(testdata_numeric)


In [32]:
#run test data through logistic regression model
finalpredictions=log_reg_classifier.predict(features_test)

In [34]:
#append is_swing output to testdata df
testdata['is_swing'] = pd.Series(finalpredictions)

In [39]:
#export testdata df to excel 
filename='finaltestdata.xlsx'
testdata.to_excel(filename)